In [ ]:
#
#   Testing MOCO methods
#

In [ ]:
# imports (using the 'moco_repair' branch of GCE)
import os
import json
import hashlib
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gcamp_extractor import MultiFileTiff
from gcamp_extractor.moco import compute_moco_offsets
from gcamp_extractor.moco import view_moco_offsets

In [ ]:
# load dataset

## three worms that drift
## can be found on YAKALO NAS under /yakalo/share/jackson/
ykroot = '/media/gbubnis/Extreme SSD/yakalo/jackson'
#root = ykroot+'/2021-07-30/worm04_act1'
#root = ykroot+'/2021-07-31/worm01_act1'  # still.. then drift + joystick correction
root = ykroot+'/2021-07-30/worm06_act1'  # head casting

params_mft = {
    'root':os.path.join(root, 'GCAMP'),
    'numz':12,
    'numc':1,
}

infojson = os.path.join(root, 'info.json')
with open(infojson) as f:
    infodict = json.load(f)

dwa = '%s_worm%02i_act%s' % (infodict['date'], int(infodict['worm']), infodict['act'])
print(dwa)

output_dir = os.path.join('mocotest_'+ dwa)
os.makedirs(output_dir, exist_ok=True)
mft = MultiFileTiff(output_dir=output_dir, **params_mft)

In [ ]:
# make param dicts
p1 = dict(mode_2D='mid', median=0, gaussian=0, method='ird', ird_filter_pcorr=0, upsample_factor=0, t_max=500)
p2 = dict(mode_2D='mid', median=5, gaussian=0, method='ird', ird_filter_pcorr=1, upsample_factor=0, t_max=500)
p3 = dict(mode_2D='mid', median=5, gaussian=2, method='ird', ird_filter_pcorr=2, upsample_factor=0, t_max=500)
p4 = dict(mode_2D='mid', median=5, gaussian=4, method='ird', ird_filter_pcorr=0, upsample_factor=0, t_max=500)
p5 = dict(mode_2D='mid', median=0, gaussian=0, method='skimage', ird_filter_pcorr=0, upsample_factor=10, t_max=500)
p6 = dict(mode_2D='mid', median=3, gaussian=0, method='skimage', ird_filter_pcorr=0, upsample_factor=10, t_max=500)
p7 = dict(mode_2D='mid', median=5, gaussian=0, method='skimage', ird_filter_pcorr=0, upsample_factor=10, t_max=500)
p8 = dict(mode_2D='mid', median=5, gaussian=4, method='skimage', ird_filter_pcorr=0, upsample_factor=10, t_max=500)
p9 = dict(mode_2D='mid', median=5, gaussian=2, method='skimage', ird_filter_pcorr=0, upsample_factor=10, t_max=500)

#NOTE: p7 is GT for /2021-07-30/worm04_act

todo = [p1, p2, p3, p4, p5, p6, p7, p8, p9]

df_todo = pd.DataFrame(todo)
df_todo.head()

In [ ]:
# compute offsets and dump to file (or load if file(s) already exists)

def get_offsets(mft, params):
    filter_keys = ['median', 'gaussian']
    other_keys = ['method', 'ird_filter_pcorr', 'upsample_factor', 't_max', 'mode_2D']
    flt = {k:params[k] for k in filter_keys}
    oth = {k:params[k] for k in other_keys}
    offsets = compute_moco_offsets(
        mft=mft,
        suppress_output=True,
        filter_params=flt,
        **oth
        )
    return offsets

all_offsets = []
for p in todo:
    xxx = json.dumps(p, sort_keys=True)
    xxx+=root
    hash8 = hashlib.sha224(str(xxx).encode('utf-8')).hexdigest()[-8:]
    
    ## already run?
    datafile = os.path.join(output_dir, 'data-%s.pk' % hash8)
    if os.path.isfile(datafile):
        print('datafile(load):', datafile)
        with open(datafile,'rb') as f:
            [p, offsets] = pickle.load(f)
    else:
        print('datafile(calc):', datafile)
        offsets = get_offsets(mft, p)
        file_pi = open(datafile, 'wb')
        pickle.dump([p, offsets], file_pi)
        file_pi.close()

    all_offsets.append(offsets)

In [ ]:
# plot offset timeseries
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(16, 10), sharey=True, sharex=True)
axs[0].axhline(y=0, color='grey')
axs[1].axhline(y=0, color='grey')

for i, (params, offsets) in enumerate(zip(todo, all_offsets)):
    xx = range(len(offsets))
    axs[0].plot(xx, np.cumsum(offsets[:,1]), '-o', color=colors[i], ms=4,  label='Y pcorr=%i' % params['ird_filter_pcorr'])
    axs[1].plot(xx, np.cumsum(offsets[:,2]), '--x', color=colors[i], ms=4, label='X pcorr=%i' % params['ird_filter_pcorr'])

axs[0].legend(fontsize='x-large')
axs[0].grid()
axs[1].legend(fontsize='x-large')
axs[1].grid()
axs[0].set_ylabel('Y offset (cumulative)', fontsize='x-large')
axs[1].set_ylabel('X offset (cumulative)', fontsize='x-large')
axs[1].set_xlabel('imaged volume, T', fontsize='x-large')
fig.suptitle(dwa, fontsize='xx-large')
fig.tight_layout()

In [ ]:
# view one case in napari (to decide what is closest to ground truth)
view_moco_offsets(mft, all_offsets[-1])